In [1]:
import random

In [2]:
result_to_id = [(i, f'id-{i}') for i in range(10)]

In [3]:
random.shuffle(result_to_id)

In [4]:
result_to_id

[(5, 'id-5'),
 (6, 'id-6'),
 (7, 'id-7'),
 (9, 'id-9'),
 (2, 'id-2'),
 (8, 'id-8'),
 (3, 'id-3'),
 (4, 'id-4'),
 (0, 'id-0'),
 (1, 'id-1')]

In [5]:
sorted(result_to_id, key=lambda x: x[0], reverse=True)

[(9, 'id-9'),
 (8, 'id-8'),
 (7, 'id-7'),
 (6, 'id-6'),
 (5, 'id-5'),
 (4, 'id-4'),
 (3, 'id-3'),
 (2, 'id-2'),
 (1, 'id-1'),
 (0, 'id-0')]

In [6]:
import numpy as np

In [7]:
results = np.array(np.arange(64))

In [8]:
results.shape

(64,)

In [9]:
results = np.reshape(results, (64, 1)).T

In [11]:
for i in results:
  print(len(i))

64
